# Resnet

{bdg-link-primary}`Paper PDF<https://arxiv.org/pdf/1512.03385>`

```{admonition} Abstract
:class: dropdown, note

Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously(vgg). We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers - 8×deeper than VGG nets [41] but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet testset. This result won the 1st place on theILSVRC 2015 classification task.  We also present analysis on CIFAR-10 with 100 and 1000 layers.The depth  of representations  is  of  central  importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement  on  the  COCO  object  detection  dataset. Deep residual nets are foundations of our submissions to ILSVRC& COCO 2015 competitions 1, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation.
```

Deep Residual Learning for Image Recognition 이란 제목의 논문으로 우리가 많이 들어본 'resnet'에 대해 나온 논문이다. 2014년에 vgg논문이 나오고 바로 다음 해에 나온 논문으로, 사실상 vgg의 구조에 residual mapping이라는 아이디어만을 추가하고도 imagenet classification에서 눈에 띄는 점수 향상을 보였다. resnet이라는 논문은 아이디어 자체가 쉽다는 점, 그리고 그 아이디어가 당시의 모델의 깊이와 모델의 성능이 선형적인 상관관계를 이루지 못하는 문제(degradation problem)를 쉬운 논리로 해결한다는 점에서 좋은 논문으로 지금까지 평가된다. 우선은 resent에서 해결하고자 하는 문제부터 살펴보자.


## 문제 설정 problem set-up

vgg를 통해서 image classification이라는 과제에서 많은 breakthrough가 있었다. cnn layer를 깊이 쌓음으로써 low/mid/high 수준의 feature들을 통합하고, 더 깊이 쌓을수록 더 많은 feature들을 data에서 추출할 수 있게 된 것이다. 하지만 무조건 깊이를 많이 쌓는다고 되었던 것은 아니다.

### 1. vanishing/exploding gradients problem

```{epigraph}
Is learning better networks as easy as stacking more layers?

-- bro. of course not!
```

첫번째 문제는 gradient가 소실되거나 폭발해버리는 문제다. layer가 몇 층 되지 않는 shallow한 network에서는 이러한 문제가 나타나지 않거나 걱정이 필요없을 정도 이지만, network가 깊어지면 gradient(경사도)가 `too small or big for training to work effectively`하게 되고 이 문제가 vanishing exploding gradient 문제다. sigmoid function [](sigmoid_function) 함수를 생각하면 문제에 대해 이애하기 쉽다.

### 2. Degradation problem


## prerequisite concepts

identity mapping



## Reference

- [Medium - Rohan #4: The vanishing gradient problem](https://ayearofai.com/rohan-4-the-vanishing-gradient-problem-ec68f76ffb9b)
- [DataScience - Neural networks: Deriving the sigmoid derivative via chain and quotient rules](https://hausetutorials.netlify.app/posts/2019-12-01-neural-networks-deriving-the-sigmoid-derivative/)